In [1]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum

# data preprocessing

In [2]:
income_df = pd.read_csv('new_income.csv')
employment_df = pd.read_csv('new_employment.csv')
population_df = pd.read_csv('new_population.csv')
facility_df = pd.read_csv('new_child_care.csv')

## (1) disctrits data with demand and population

In [3]:
merged_df = pd.merge(income_df, employment_df, on = 'zip_code')

def classify_demand(row):
    if row['employment rate'] >= 0.6 or row['average income'] <= 60000:
        return 'High-Demand'
    else:
        return 'Normal-Demand'

merged_df['Demand'] = merged_df.apply(classify_demand, axis=1)

merged_df = pd.merge(merged_df, population_df, on = 'zip_code')

merged_df['p_0-5'] = merged_df['-5']
merged_df['p_5-12'] = np.floor(merged_df['5-9'] + 3/5 * merged_df['10-14'])
merged_df['p_0-12'] = merged_df['p_0-5'] + merged_df['p_5-12']

df = merged_df[['zip_code' , 'Demand', 'p_0-5', 'p_5-12', 'p_0-12']]

df.head()

,zip_code,Demand,p_0-5,p_5-12,p_0-12
0,10001,Normal-Demand,744,1349.0,2093.0
1,10002,High-Demand,2142,4964.0,7106.0
2,10003,Normal-Demand,1440,1605.0,3045.0
3,10004,Normal-Demand,433,278.0,711.0
4,10005,High-Demand,484,341.0,825.0


## (2) facilities data with id and capacity

In [4]:
facility_df['c_0-5'] = facility_df['infant_capacity'] + facility_df['toddler_capacity'] + facility_df['preschool_capacity'] + np.floor(5/12 * facility_df['children_capacity'])
facility_df['c_5-12'] = np.floor(7/12 * facility_df['children_capacity'])
facility_df['c_0-12'] = facility_df['c_0-5'] + facility_df['c_5-12']

facility_df = facility_df[['zip_code' ,'facility_id', 'c_0-5', 'c_5-12', 'c_0-12']]

facility_df.head()

,zip_code,facility_id,c_0-5,c_5-12,c_0-12
0,10001,837597,0.0,0.0,0.0
1,10001,661697,5.0,7.0,12.0
2,10001,837329,0.0,0.0,0.0
3,10001,350076,2.0,3.0,5.0
4,10001,292419,0.0,0.0,0.0


## (3) aggregate of facilities data in every districts

In [5]:
facility = facility_df.groupby('zip_code').agg({
    'c_0-5': 'sum',
    'c_5-12': 'sum',
    'c_0-12': 'sum'
}).reset_index()

facility.head()

,zip_code,c_0-5,c_5-12,c_0-12
0,10001,9.0,13.0,22.0
1,10002,95.0,108.0,203.0
2,10003,0.0,0.0,0.0
3,10004,0.0,0.0,0.0
4,10005,0.0,0.0,0.0


## (4) districts data with demand, population and capacity

In [6]:
df = pd.merge(df, facility, on = 'zip_code')
df.head()

,zip_code,Demand,p_0-5,p_5-12,p_0-12,c_0-5,c_5-12,c_0-12
0,10001,Normal-Demand,744,1349.0,2093.0,9.0,13.0,22.0
1,10002,High-Demand,2142,4964.0,7106.0,95.0,108.0,203.0
2,10003,Normal-Demand,1440,1605.0,3045.0,0.0,0.0,0.0
3,10004,Normal-Demand,433,278.0,711.0,0.0,0.0,0.0
4,10005,High-Demand,484,341.0,825.0,0.0,0.0,0.0


# question 1

In [12]:
# Initialize the model
model = Model('ChildCareOptimization')

facility_sizes = {
    'Small': {'TotalSlots': 100, 'Under5Slots': 50, 'Cost': 65000},
    'Medium': {'TotalSlots': 200, 'Under5Slots': 100, 'Cost': 95000},
    'Large': {'TotalSlots': 400, 'Under5Slots': 200, 'Cost': 115000}
}

under5_equipment_cost = 100

I = {}  # Additional slots through expansion at each facility
U = {}  # Under-5 slots added through expansion at each facility
N = {}  # Number of new facilities built in each zip code

# **Initialize the cost components outside the loop**
expansion_costs = []
construction_costs = []

for index, row in df.iterrows():
    z = row['zip_code']
    demand_class = row['Demand']
    population_0_5 = row['p_0-5']
    population_0_12 = row['p_0-12']
    existing_slots_0_5 = row['c_0-5']
    existing_slots_0_12 = row['c_0-12']

    # Get facilities in zip code z
    facilities_in_zip = facility_df[facility_df['zip_code'] == z]

    # Initialize lists to collect variables for constraints per zip code
    expansion_slots = []
    under5_expansion_slots = []

    # Decision variables for expansions at each facility in zip code z
    for idx, facility in facilities_in_zip.iterrows():
        f = facility['facility_id']
        current_capacity = facility['c_0-12']
        current_under5 = facility['c_0-5']

        if current_capacity > 0:
            max_expansion = min(0.2 * current_capacity, 500)

            # Decision variables for expansion at facility f
            I[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_expansion, name=f'I_{z}_{f}')
            U[z, f] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=max_expansion, name=f'U_{z}_{f}')

            # Constraint: U[z, f] <= I[z, f]
            model.addConstr(I[z, f] - U[z, f] >= 0, name=f'Under5ExpansionLimit_{z}_{f}')

            # Collect variables for constraints
            expansion_slots.append(I[z, f])
            under5_expansion_slots.append(U[z, f])

            # **Calculate expansion cost for facility f and add to the list**
            base_cost = 20000 + 200 * current_capacity
            expansion_costs.append(
                (I[z, f] / current_capacity) * base_cost + under5_equipment_cost * U[z, f]
            )

        else:
            # Facility with zero capacity cannot be expanded
            continue

    # Decision variables for new facilities in zip code z
    for s in facility_sizes.keys():
        N[z, s] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f'N_{z}_{s}')

        # **Calculate construction cost for new facilities and add to the list**
        facility_info = facility_sizes[s]
        construction_costs.append(
            N[z, s] * (facility_info['Cost'] + under5_equipment_cost * facility_info['Under5Slots'])
        )

    # Calculate required slots based on demand classification
    if demand_class == 'High-Demand':
        required_total_slots = 0.5 * population_0_12
    else:
        required_total_slots = (1/3) * population_0_12

    required_under5_slots = (2/3) * population_0_5

    # Total existing slots (including expansions)
    total_existing_slots = existing_slots_0_12
    total_existing_under5_slots = existing_slots_0_5

    # Total expansion slots
    total_expansion_slots = quicksum(expansion_slots)
    total_under5_expansion_slots = quicksum(under5_expansion_slots)

    # Total new slots from new facilities
    total_new_slots = quicksum(N[z, s] * facility_sizes[s]['TotalSlots'] for s in facility_sizes.keys())
    total_new_under5_slots = quicksum(N[z, s] * facility_sizes[s]['Under5Slots'] for s in facility_sizes.keys())

    # Constraint: Total slots requirement
    model.addConstr(
        total_existing_slots + total_expansion_slots + total_new_slots >= required_total_slots,
        name=f'TotalSlotsRequirement_{z}'
    )

    # Constraint: Under-5 slots requirement
    model.addConstr(
        total_existing_under5_slots + total_under5_expansion_slots + total_new_under5_slots >= required_under5_slots,
        name=f'Under5SlotsRequirement_{z}'
    )

# **Set the objective function outside the loop**
model.setObjective(
    quicksum(expansion_costs) + quicksum(construction_costs),
    GRB.MINIMIZE
)

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    total_cost = model.objVal
    print(f'Total Minimum Cost: ${total_cost:,.2f}\n')

    # Display expansions at each facility
    print('Facility Expansions:')
    for var in model.getVars():
        if 'I_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')

    # Display under-5 slots added in expansions
    print('\nUnder-5 Slots in Expansions:')
    for var in model.getVars():
        if 'U_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')

    # Display new facilities to be built
    print('\nNew Facilities to be Built:')
    for var in model.getVars():
        if 'N_' in var.varName and var.x > 0:
            print(f'{var.varName} = {var.x}')
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 14199 rows, 27375 columns and 54750 nonzeros
Model fingerprint: 0x4a217140
Variable types: 0 continuous, 27375 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [1e+00, 9e+01]
  RHS range        [5e-01, 1e+04]
Found heuristic solution: objective 4.365900e+08
Presolve removed 13935 rows and 26848 columns
Presolve time: 0.75s
Presolved: 264 rows, 527 columns, 1054 nonzeros
Found heuristic solution: objective 3.991983e+08
Variable types: 0 continuous, 527 integer (122 binary)

Root relaxation: objective 3.991489e+08, 317 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl